Create plot of the SLA L4 product.

In [53]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib import rcParams
import netCDF4
import logging

In [25]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [69]:
rcParams.update({'font.size': 14, 'figure.dpi': 300, 'savefig.bbox': 'tight'})

# User inputs

## Domain

In [26]:
coordinates = [-6.75, 40.001, 30, 48.]

## Data

We downloaded the product *SEALEVEL_MED_SLA_MAP_L4_REP_OBSERVATIONS_008_029* from [CMEMS](http://marine.copernicus.eu).<br>
It corresponds to *dataset-duacs-rep-medsea-merged-allsat-msla-l4*: a reprocessed product of Sea Level Anomalies (SLA) using all the available satellites

In [76]:
datadir = os.path.expanduser("~/Data/AVISO/Gridded/2014")
figdir = os.path.expanduser("~/Data/AVISO/Gridded/figures/2014")
logging.info("Data directory: {0}".format(datadir)) 

INFO:root:Data directory: /home/ctroupin/Data/AVISO/Gridded/2014


# File list

In [14]:
datafilelist = sorted(glob.glob(os.path.join(datadir, "dt_med_allsat_msla_h_*.nc")))
logging.info("Working on {0} files".format(len(datafilelist)))

INFO:root:Working on 365 files


In [74]:
def readGriddedGSLA(datafile):
    """
    Read the SLA and the coordinates from a netCDf file
    Note that longitute have to be transformed in order to lie beween -180 and 180.
    """
    with netCDF4.Dataset(datafile, 'r') as nc:
        ttime = nc.variables['time'][:]
        timeunits = nc.variables['time'].units
        lat = nc.variables['lat'][:]
        lon = nc.variables['lon'][:]
        lon -= 360.
        sla = nc.variables['sla'][:]
        error = nc.variables['err'][:]
    filetime = netCDF4.num2date(ttime, timeunits)
    # Get date from netCDF variable `time`
    return ttime, lon, lat, sla, error, filetime

In [88]:
def plotGriddedSLA(lon, lat, sla, figname, figtitle):
    """
    Create pcolor plot of the Sea Level Anomaly field
    and generate a figure
    """
    
    # Prepare data
    llon, llat = np.meshgrid(lon, lat)
    sla = np.squeeze(sla)
                 
    # Prepare figure
    fig = plt.figure(figsize=(6, 6))
    m.pcolormesh(llon, llat, sla, latlon=True, cmap=plt.cm.RdYlBu_r, vmin=-.2, vmax=.2)
    # Add lines, coastline etc    
    m.drawmeridians(np.arange(-8., round(coordinates[1]), 8), 
                    labels=[0, 0, 0, 1], linewidth=.2)
    m.drawparallels(np.linspace(round(coordinates[2]), round(coordinates[3]), 5), 
                    labels=[1, 0, 0, 0], linewidth=.2)
    m.drawcoastlines(linewidth=.2)
    cbar = plt.colorbar(shrink=.5, extend='both')
    cbar.set_label("m", fontsize=12, rotation=0)
    plt.title(figtitle)
    plt.savefig(figname, dpi=300)
    #plt.show()
    plt.close()

## Projection

In [89]:
m = Basemap(projection='merc',
            llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
            urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
            lat_ts=0.5 * (coordinates[2] + coordinates[3]), resolution='i')

## Loop on files

In [91]:
for datafile in datafilelist[100:151]:
    fname = os.path.basename(datafile)
    figname = fname.split('.')[0]
    logging.debug("Working on {0}".format(fname))
    
    # Read data
    ttime, lon, lat, sla, error, filetime = readGriddedGSLA(datafile)
    # Plot
    figtitle = filetime[0].strftime("%Y-%m-%d")
    plotGriddedSLA(lon, lat, sla, os.path.join(figdir, figname), figtitle)

DEBUG:root:Working on dt_med_allsat_msla_h_20140411_20140829.nc
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3413: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3422: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)
DEBUG:root:Working on dt_med_allsat_msla_h_20140412_20140829.nc
DEBUG:root:Working on dt_med_allsat_msla_h_20140413_20140829.nc
DEBUG:root:Working on dt_med_allsat_msla_h_20140414_20140829.nc
DEBUG:root:Working on dt_med_allsat_msla_h_20140415_20140829.nc
DEBUG:root:Working on dt_med_allsat_msla_h_20140416_20140829.nc
DEBUG:root:Working on dt_med_allsat_msla_h_20140417_20140829.nc
DEBUG:root:Working on dt_med_allsat_msla_h_20140418